In [99]:
import json
import os
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt

# path to json file that stores MFCCs and genre labels for each processed segment
DATA_PATH = os.path.join(os.getcwd(), 'mfcc')
print(DATA_PATH)

/home/asif/Documents/Project/openmic-2018/mfcc


In [100]:
def load_data(data_path):
    """Loads training dataset from json file.
        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    # convert lists to numpy arrays
    X = np.array(data["mfccs"])
    #y = np.array(data["labels"])

    print("Data succesfully loaded!")

    return  X

In [101]:
MFCC_PATH = os.path.join(os.getcwd(), 'mfcc')
mfccs = []
for path, subdirs, files in os.walk(MFCC_PATH):
    dir_name = path.split('/')[-1]
    for f in files:
        mfcc_path = os.path.join(path, f)
        sample_key = f.split('.')[0]
        # print(sample_key)
        with open(mfcc_path, 'r') as mfcc_data:
            json_mfcc = json.load(mfcc_data)
            mfccs.append(json_mfcc['mfcc'])
    if dir_name == '000':
        break


In [102]:
mfcc_data = np.array(mfccs)
#mfcc_data = np.array(mfccs).reshape(132,431,13)
print(mfcc_data.shape)

(2692,)


In [103]:
def get_key(my_dict, val):
    for key, value in my_dict.items():
        if val == value:
            return key
 
    return "key doesn't exist"

In [104]:
with open("/home/asif/Documents/Project/openmic-2018/class-map.json", "r") as fp:
    label_map = json.load(fp)
#label_map = np.array(data_map)


In [105]:
instrument = get_key(label_map, 2)
print(instrument)

bass


In [106]:
#loadX= load_data("/home/asif/Documents/Project/openmic-2018/mfcc/000/000046_3840.json")

loadX_new = load_data("/home/asif/Documents/Project/openmic-2018/mfccs.json")

print(loadX_new.shape)

Data succesfully loaded!
(132, 431, 13)


In [107]:
OPENMIC = np.load(os.path.join(os.getcwd(), 'openmic-2018.npz'))
print(list(OPENMIC.keys()))

['X', 'Y_true', 'Y_mask', 'sample_key']


In [108]:
Y_true = np.array(OPENMIC['Y_mask'])
Y = Y_true[:132]
print(Y.shape)
len = Y.shape[0]
Y_one = []
Y_final = []
for i in range(len):
    Y_one.append(np.where(Y[i]))
    Y_final.append(Y_one[i][0][0])
Y_final = np.array(Y_final)
#Y_final = Y_final.reshape(-1)
#print(Y_one)
print(Y)

Y = Y.astype(int)
print(Y)
print(load_X_vgg.shape)

(132, 20)
[[False False False ... False False False]
 [False False False ... False False  True]
 [ True False False ... False False False]
 ...
 [False False False ... False False  True]
 [False False False ... False False False]
 [False False False ... False False False]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(132, 10, 128)


In [109]:
def build_model(input_shape, classes):
    """Generates CNN model
    :param input_shape (tuple): Shape of input set
    :classes: shape of output
    :return model: CNN model
    """

    # build network topology
    model = keras.Sequential()

    # 1st conv layer
    model.add(keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 2nd conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 3rd conv layer
    model.add(keras.layers.Conv2D(64, (2, 2), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # flatten output and feed it into dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    #model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(64, activation='relu'))
    #model.add(keras.layers.Dropout(0.3))

    # output layer
    model.add(keras.layers.Dense(classes, activation='sigmoid'))
    #model.add(Dense(classes))
    #model.add(Activation(finalAct))

    return model

In [110]:
def predict(model, X):
    """Predict a single sample using the trained model
    :param model: Trained classifier
    :param X: Input data
    :param y (int): Target
    """

    # add a dimension to input data for sample - model.predict() expects a 4d array in this case
    X = X[np.newaxis, ...] # array shape (1, 130, 13, 1)

    # perform prediction
    prediction = model.predict(X)
    #print(prediction)
    # get index with max value
    predicted_index = np.argsort(prediction[0])[:-11:-1] 
    #predicted_index = np.argmax(prediction, axis=1)
    for i in range(10):
        instrument = get_key(label_map, predicted_index[i])
        print("{}".format(instrument)+" ({:.3})".format(prediction[0][predicted_index[i]]))
    
    #print("Target: {}, Predicted label: {}".format(y, instrument))
    

In [111]:
X = loadX_new
#X = load_X_vgg
test_size, validation_size = 0.25, 0.2
print(X.shape[0], X.shape[1])
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size)
#X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

# add an axis to input sets
X_train = X_train[..., np.newaxis]
#X_validation = X_validation[..., np.newaxis]
X_test = X_test[..., np.newaxis]

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape )

132 431
(99, 431, 13, 1) (33, 431, 13, 1) (99, 20) (33, 20)


In [112]:
input_shape = (X_train.shape[1], X_train.shape[2], 1)
#input_shape = (431, 13, 1)
print(input_shape)
model = build_model(input_shape, Y.shape[1])

(431, 13, 1)


In [120]:
optimiser = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimiser,
              loss="binary_crossentropy",
              metrics=['accuracy'])

model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 429, 11, 16)       160       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 215, 6, 16)        0         
_________________________________________________________________
batch_normalization_26 (Batc (None, 215, 6, 16)        64        
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 213, 4, 32)        4640      
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 107, 2, 32)        0         
_________________________________________________________________
batch_normalization_27 (Batc (None, 107, 2, 32)        128       
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 106, 1, 64)      

In [118]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape )
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=50)


(99, 431, 13, 1) (33, 431, 13, 1) (99, 20) (33, 20)
Epoch 1/50
4/4 [==============================] - 0s 117ms/step - loss: 0.0042 - accuracy: 0.6566 - val_loss: 0.4510 - val_accuracy: 0.0606
Epoch 2/50
4/4 [==============================] - 0s 72ms/step - loss: 0.0045 - accuracy: 0.6364 - val_loss: 0.4541 - val_accuracy: 0.0606
Epoch 3/50
4/4 [==============================] - 0s 52ms/step - loss: 0.0071 - accuracy: 0.6364 - val_loss: 0.4571 - val_accuracy: 0.0606
Epoch 4/50
4/4 [==============================] - 0s 46ms/step - loss: 0.0048 - accuracy: 0.6162 - val_loss: 0.4607 - val_accuracy: 0.0606
Epoch 5/50
4/4 [==============================] - 0s 43ms/step - loss: 0.0049 - accuracy: 0.6465 - val_loss: 0.4641 - val_accuracy: 0.0606
Epoch 6/50
4/4 [==============================] - 0s 50ms/step - loss: 0.0046 - accuracy: 0.6263 - val_loss: 0.4670 - val_accuracy: 0.0606
Epoch 7/50
4/4 [==============================] - 0s 49ms/step - loss: 0.0052 - accuracy: 0.6263 - val_loss: 0.46

In [115]:
X_to_predict = X_test[9]
y_to_predict = y_test[9]

# predict 10 possible instruments
predict(model, X_to_predict)

accordion (0.101)
violin (0.0846)
mandolin (0.0659)
clarinet (0.0645)
ukulele (0.0511)
trumpet (0.0452)
saxophone (0.0358)
mallet_percussion (0.0334)
voice (0.0215)
banjo (0.0128)


In [116]:
#print(y_to_predict.shape)
for i in range(20):
    if y_to_predict[i] == 1:
        print(get_key(label_map, i))

accordion
clarinet


In [191]:
Y_mask = np.array(OPENMIC['Y_mask'])
for instrument in label_map:
    
    # Map the instrument name to its column number
    inst_num = label_map[instrument]
        
    # Step 1: sub-sample the data
    
    # First, we need to select down to the data for which we have annotations
    # This is what the mask arrays are for
    train_inst = Y_mask[:, inst_num]
    print(train_inst)


[False False  True ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[ True False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[ True False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False False False ... False False False]
[False  True False ...  True  True  True]
[False False False ... False False False]
[False False False ...  True False  True]
[ True  True False ... False False  True]
[False False False ... False False False]
[False False False ... False False False]
[False  True False ... False False False]


In [119]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

2/2 - 0s - loss: 0.5789 - accuracy: 0.0909

Test accuracy: 0.09090909361839294
